# 非線形パネルVAR-SPDE-LSTMモデル(精度評価付き)

# 著者
- 氏名: 柴田怜
- 職業: Data Scientist
- 学位: 上智大学大学院修士(環境学)・学士(経済学)

## 前提
1. メモリ容量の最大化
1. 必要なライブラリの読込
1. 関数定義
1. ローデータの目視確認

## 実行手順
1. 誤差項調整
1. 多重共線性の実証分析
1. 無相関検定
1. 単位根検定(ADF検定)
1. 共和分検定
1. 偏グレンジャー因果性検定と非直交化インパルス応答関数
1. パネルVARモデルによる動的直接相関係数の導出
1. 幾何ブラウン運動
1. LSTM(Long Short Term Memory)
1. グラフ描画・出力

## メモリ容量の最大化

In [1]:
memory.limit(memory.size(max = T))
gc(verbose = getOption("verbose"), reset = T, full = T)

Warning message in memory.limit(memory.size(max = T)):
" メモリー限界を減らすことができません: 無視しました "


[1] 32176

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,524108,28.0,1178095,63,524108,28.0
Vcells,961226,7.4,8388608,64,961226,7.4


## 必要なライブラリの読込

In [39]:
load.lib <- c(
 "data.table"     #.csv読込・出力
 , "magrittr"     #前処理
 , "tidyr"        #前処理
 , "dplyr"        #前処理
 , "tidyverse"    #前処理
 , "tseries"      #前処理
 , "urca"         #ADF検定
 , "aTSA"         #共和分検定
 , "plm"          #パネルデータの形成
 , "panelvar"     #パネルVARモデル
 , "Sim.DiffProc" #幾何ブラウン運動
 , "ggplot2"      #可視化
 , "gridExtra"    #グラフの集約
 , "qgraph"       #相関行列の可視化
 , "tsbox"        #ts_df関数の利用
 , "keras"        #LSTMによる幾何ブラウン運動の精度評価
 , "tensorflow"   #LSTMによる幾何ブラウン運動の精度評価
 , "vars"         #関数定義に用いる(以下、同じ)。
 , "NlinTS"
 , "tsDyn"
)
install.lib <- load.lib[!load.lib %in% installed.packages()]
for(lib in install.lib) install.packages(lib,dependencies = T)
sapply(load.lib,require,character = T)
#LSTMによる幾何ブラウン運動の精度評価
install_tensorflow(gpu = TRUE) #GPUの利用

data.table     magrittr        tidyr        dplyr    tidyverse      tseries 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
        urca         aTSA          plm     panelvar Sim.DiffProc      ggplot2 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
   gridExtra       qgraph        tsbox        keras   tensorflow         vars 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
      NlinTS        tsDyn 
        TRUE         TRUE

## 関数定義
- 対数差分系列
- プロビット写像
- 偏グレンジャー因果性検定と非直交化インパルス応答関数
- ADF検定
- 標本分散
- 時系列プロット
- LSTM(Long Short Term Memory)

In [3]:
#対数差分系列に変換する。
diff.log <- function(x) {
  y <- diff(log(x))
  return(y)
}
#プロビット写像
probit <- function(x) {
  y <- c(exp(((-x^2)/2))/sqrt(2 * pi))
  return(y)
}
#偏グレンジャー因果性検定と非直交化インパルス応答関数
ts <- function(y1, y2) {
  temp  <- cbind(y1, y2) %>% as.data.frame
  model <- VAR(temp, p = 2, type = "both", ic = "AIC")
  wk_result_1  <- causality(model, cause = "y1")
  wk_result_2  <- causality(model, cause = "y2")
  granger      <- list(wk_result_1, wk_result_2)
  impulse_1 <- irf(model, impulse = "y1", response = "y2", boot = F)
  impulse_2 <- irf(model, impulse = "y2", response = "y1", boot = F)
  imp <- list(impulse_1, impulse_2)
  result <- list(granger, imp)
  return(result)
}
#ADF検定
ADF <- function(x) {
  result <- ur.df(x,type = c("drift"), lags = 1) %>%
    summary
  return(result)
}
#標本分散
sigma <- function(x) {
  result <- var(x)*(length(x)-1)/length(x)
  return(result)
}
#時系列プロット
fig <- function(data, y, title, label) {
  data %>%
    ggplot(aes(x = time, y = y)) +
    geom_point() +
    geom_line() +
    ggtitle(title) +
    labs(x = "年", y =  label)
}
#LSTM(Long Short Term Memory)の前処理
#時系列データts_dfをwindowで指定した長さ毎に区切って訓練用のデータを生成する。
LSTM <- function(ts_df, window, rm.na = F) {
  data.x = NULL
  data.y = NULL
  n = dim(ts_df)[2]
  for(i in 1:n){
    ts_x = ts_df[,i]
    for(j in 1:(length(ts_x)-window)){
      if(rm.na){
        tmp.x = ts_x[1:window + j -1]
        tmp.y = ts_x[1:window + j -1]
        if(sum(c(is.na(tmp.x),is.na(tmp.y))) == 0){
          data.x = rbind(data.x,ts_x[1:window + j -1])
          data.y = rbind(data.y,ts_x[window + j])          
        }
      }else{
        data.x = rbind(data.x,ts_x[1:window + j -1])
        data.y = rbind(data.y,ts_x[window + j])
      }
    }
  }
  data <- list(x = array_reshape(data.x,c(dim(data.x),1)),
               y = data.y)
  scale = max(ts_df, na.rm = T)
  x = data$x / scale
  y = data$y / scale
  lstm <- keras_model_sequential()
  lstm %>%
    layer_lstm(units = 64,input_shape = c(dim(x)[2],1)) %>%
    layer_dropout(rate = 0.4) %>%
    layer_dense(units = 1)
  lstm %>% compile(loss = "mean_squared_error",
                   optimizer = optimizer_adam(),
                   metrics = "accuracy")
  lstm %>% fit(x,y,
               epochs = 1000,batch_size = 10, validation_split = 0.2)
  i = 1
  test_x = ts_df[,i]
  test_x = data$x
  scale=max(ts_df,na.rm = T)
  test_x = test_x/scale
  pred_x = lstm %>% 
    predict(test_x)  
  ts.plot(ts_df[,i]/scale->a,ylim = c(min(c(a,pred_x)), max(c(a,pred_x))), ylab = "検証値")
  lines(c(rep(NA,window),pred_x), col = 2)
}

# ローデータの目視確認
- 読込
- 要約統計量を求める。

In [4]:
#読込
raw_data <- fread("./0_input/raw_data.csv") %>%
  as.data.frame
raw_data

Warning message in require_bit64_if_needed(ans):
"Some columns are type 'integer64' but package bit64 is not installed. Those columns will print as strange looking floating point data. There is no need to reload the data. Simply install.packages('bit64') to obtain the integer64 print method and print the data again."


id,time,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11
<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int64>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0,1986,11.36838,1111250,0,20.90,1.212152e-313,42.4,85.9,2.6,217.71013,58.594,0
0,1987,11.91077,1111019,0,24.12,1.437787e-313,41.9,85.9,2.8,251.03609,58.594,0
0,1988,12.58026,1186750,0,18.37,1.734345e-313,40.4,86.5,2.8,348.65896,58.594,0
0,1989,13.02217,1216857,0,22.40,1.735299e-313,38.7,88.5,2.5,439.72734,58.594,0
0,1990,13.54005,1304676,0,27.67,1.772752e-313,36.8,91.2,2.3,261.49901,53.711,0
0,1991,13.73636,1318934,1,23.65,1.974477e-313,36.2,94.3,2.1,243.73033,53.711,0
0,1992,13.85659,1330643,0,22.79,2.087208e-313,36.9,95.8,2.1,176.66962,43.640,1
0,1993,14.00667,1333073,0,19.51,2.313130e-313,39.9,97.1,2.2,179.37425,43.640,0
0,1994,14.50500,1385835,0,18.97,2.530688e-313,41.1,97.7,2.5,201.87369,43.640,1


In [5]:
# 要約統計量を求める。
raw_data %>%
  summary

       id              time            Y1              Y2         
 Min.   :0.0000   Min.   :1986   Min.   :11.37   Min.   :1111019  
 1st Qu.:0.0000   1st Qu.:1993   1st Qu.:13.86   1st Qu.:1315869  
 Median :0.0000   Median :2000   Median :14.50   Median :1385835  
 Mean   :0.1379   Mean   :2000   Mean   :14.44   Mean   :1352237  
 3rd Qu.:0.0000   3rd Qu.:2007   3rd Qu.:15.46   3rd Qu.:1422453  
 Max.   :1.0000   Max.   :2014   Max.   :15.74   Max.   :1478859  
       Y3               Y4              Y5                   Y6        
 Min.   :0.0000   Min.   :15.90   Min.   :1.212e-313   Min.   : 36.20  
 1st Qu.:0.0000   1st Qu.:22.40   1st Qu.:2.052e-313   1st Qu.: 41.90  
 Median :0.0000   Median :31.80   Median :2.303e-313   Median : 69.50  
 Mean   :0.2069   Mean   :44.78   Mean   :2.288e-313   Mean   : 77.77  
 3rd Qu.:0.0000   3rd Qu.:71.49   3rd Qu.:2.531e-313   3rd Qu.:102.00  
 Max.   :1.0000   Max.   :95.30   Max.   :3.176e-313   Max.   :149.50  
       Y7               Y8 

## 誤差項調整
- 対数差分系列(一次のテイラー展開近似によって変化率に近似するとともに、定常状態として扱う為)
- プロビット写像(Y3: 原子力事故・異常事象(有無)及びY11: 気候変動対策に関する合意(有無))

In [6]:
adjusted <- bind_cols(
    raw_data$id[-1]
    , time = raw_data$time[-1]
    , Y1   = diff.log(raw_data$Y1)
    , Y2   = diff.log(raw_data$Y2)
    , Y3   = probit(raw_data$Y3) %>% diff.log
    , Y4   = diff.log(raw_data$Y4)
    , Y5   = diff.log(raw_data$Y5)
    , Y6   = diff.log(raw_data$Y6)
    , Y7   = diff.log(raw_data$Y7)
    , Y8   = diff.log(raw_data$Y8)
    , Y9   = diff.log(raw_data$Y9)
    , Y10  = diff.log(raw_data$Y10)
    , Y11  = probit(raw_data$Y11) %>% diff.log
) %>%
  as.data.frame %>%
  apply(2, as.numeric)
#列名を戻す。
colnames(adjusted) <- colnames(raw_data)
#目視確認
adjusted

New names:
* `` -> ...1



id,time,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11
0,1987,0.046607215,-0.0002075447,0.0,0.14329221,0.1707077926,-0.011862535,0.000000000,0.07410797,0.142432217,0.0000000000,0.0
0,1988,0.054685886,0.0659405832,0.0,-0.27232247,0.1875249212,-0.036456042,0.006960585,0.00000000,0.328497543,0.0000000000,0.0
0,1989,0.034524371,0.0250528768,0.0,0.19834206,0.0005498574,-0.042990185,0.022858138,-0.11332869,0.232060612,0.0000000000,0.0
0,1990,0.038998803,0.0696837388,0.0,0.21128783,0.0213535400,-0.050341755,0.030052345,-0.08338161,-0.519724354,-0.0870144800,0.0
0,1991,0.014394272,0.0108689107,-0.5,-0.15698568,0.1077701561,-0.016438726,0.033426293,-0.09097178,-0.070368106,0.0000000000,0.0
0,1992,0.008714722,0.0088388117,0.5,-0.03704127,0.0555237206,0.019152432,0.015781495,0.00000000,-0.321780945,-0.2076436613,-0.5
0,1993,0.010772766,0.0018240989,0.0,-0.15539469,0.1027744168,0.078164773,0.013478690,0.04652002,0.015192965,0.0000000000,0.5
0,1994,0.034959424,0.0388164337,0.0,-0.02806837,0.0898898827,0.029631798,0.006160184,0.12783337,0.118167806,0.0000000000,-0.5
0,1995,0.031159255,0.0255087629,-0.5,-0.01487016,0.0913039761,0.059049029,-0.001024066,0.14842001,0.008353503,0.0000000000,0.5
0,1996,0.015857401,0.0017371782,0.5,0.19217123,-0.1259988187,0.060084811,0.001024066,0.09844007,-0.026863202,0.0868586003,0.0


## 多重共線性の実証分析

In [7]:
#確率変数のみのデータフレーム
relation <- adjusted[, !(colnames(adjusted) %in% c("id", "time"))] %>%
  apply(2, as.numeric) %>%
  as.data.frame
#目視確認
relation

Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.046607215,-0.0002075447,0.0,0.14329221,0.1707077926,-0.011862535,0.000000000,0.07410797,0.142432217,0.0000000000,0.0
0.054685886,0.0659405832,0.0,-0.27232247,0.1875249212,-0.036456042,0.006960585,0.00000000,0.328497543,0.0000000000,0.0
0.034524371,0.0250528768,0.0,0.19834206,0.0005498574,-0.042990185,0.022858138,-0.11332869,0.232060612,0.0000000000,0.0
0.038998803,0.0696837388,0.0,0.21128783,0.0213535400,-0.050341755,0.030052345,-0.08338161,-0.519724354,-0.0870144800,0.0
0.014394272,0.0108689107,-0.5,-0.15698568,0.1077701561,-0.016438726,0.033426293,-0.09097178,-0.070368106,0.0000000000,0.0
0.008714722,0.0088388117,0.5,-0.03704127,0.0555237206,0.019152432,0.015781495,0.00000000,-0.321780945,-0.2076436613,-0.5
0.010772766,0.0018240989,0.0,-0.15539469,0.1027744168,0.078164773,0.013478690,0.04652002,0.015192965,0.0000000000,0.5
0.034959424,0.0388164337,0.0,-0.02806837,0.0898898827,0.029631798,0.006160184,0.12783337,0.118167806,0.0000000000,-0.5
0.031159255,0.0255087629,-0.5,-0.01487016,0.0913039761,0.059049029,-0.001024066,0.14842001,0.008353503,0.0000000000,0.5


## 無相関検定
- 相関係数に統計的有意性を検出すれば、多重共線性がある。

In [8]:
#Y1～11
cor.test(relation$Y1, relation$Y2,   method = "pearson")
cor.test(relation$Y1, relation$Y3,   method = "pearson")
cor.test(relation$Y1, relation$Y4,   method = "pearson") 
cor.test(relation$Y1, relation$Y5,   method = "pearson")  
cor.test(relation$Y1, relation$Y6,   method = "pearson")  
cor.test(relation$Y1, relation$Y7,   method = "pearson")  
cor.test(relation$Y1, relation$Y8,   method = "pearson")  
cor.test(relation$Y1, relation$Y9,   method = "pearson")  
cor.test(relation$Y1, relation$Y10,  method = "pearson")  
cor.test(relation$Y1, relation$Y11,  method = "pearson") 
#Y2～11
cor.test(relation$Y2, relation$Y3,   method = "pearson")  
cor.test(relation$Y2, relation$Y4,   method = "pearson")  
cor.test(relation$Y2, relation$Y5,   method = "pearson")  
cor.test(relation$Y2, relation$Y6,   method = "pearson")  
cor.test(relation$Y2, relation$Y7,   method = "pearson")  
cor.test(relation$Y2, relation$Y8,   method = "pearson")  
cor.test(relation$Y2, relation$Y9,   method = "pearson")  
cor.test(relation$Y2, relation$Y10,  method = "pearson")  
cor.test(relation$Y2, relation$Y11,  method = "pearson")
#Y3～11
cor.test(relation$Y3, relation$Y4,   method = "pearson")  
cor.test(relation$Y3, relation$Y5,   method = "pearson")  
cor.test(relation$Y3, relation$Y6,   method = "pearson")  
cor.test(relation$Y3, relation$Y7,   method = "pearson")  
cor.test(relation$Y3, relation$Y8,   method = "pearson")  
cor.test(relation$Y3, relation$Y9,   method = "pearson")  
cor.test(relation$Y3, relation$Y10,  method = "pearson")  
cor.test(relation$Y3, relation$Y11,  method = "pearson")
#Y4～11
cor.test(relation$Y4, relation$Y5,   method = "pearson")  
cor.test(relation$Y4, relation$Y6,   method = "pearson")  
cor.test(relation$Y4, relation$Y7,   method = "pearson")  
cor.test(relation$Y4, relation$Y8,   method = "pearson")  
cor.test(relation$Y4, relation$Y9,   method = "pearson")  
cor.test(relation$Y4, relation$Y10,  method = "pearson")  
cor.test(relation$Y4, relation$Y11,  method = "pearson")
#Y5～11
cor.test(relation$Y5, relation$Y6,   method = "pearson")  
cor.test(relation$Y5, relation$Y7,   method = "pearson")  
cor.test(relation$Y5, relation$Y8,   method = "pearson")  
cor.test(relation$Y5, relation$Y9,   method = "pearson")  
cor.test(relation$Y5, relation$Y10,  method = "pearson")  
cor.test(relation$Y5, relation$Y11,  method = "pearson")
#Y6～11
cor.test(relation$Y6, relation$Y7,   method = "pearson")  
cor.test(relation$Y6, relation$Y8,   method = "pearson")  
cor.test(relation$Y6, relation$Y9,   method = "pearson")  
cor.test(relation$Y6, relation$Y10,  method = "pearson")  
cor.test(relation$Y6, relation$Y11,  method = "pearson")
#Y7～11
cor.test(relation$Y7, relation$Y8,   method = "pearson")  
cor.test(relation$Y7, relation$Y9,   method = "pearson")  
cor.test(relation$Y7, relation$Y10,  method = "pearson")  
cor.test(relation$Y7, relation$Y11,  method = "pearson")
#Y8～11
cor.test(relation$Y8, relation$Y9,   method = "pearson")  
cor.test(relation$Y8, relation$Y10,  method = "pearson")  
cor.test(relation$Y8, relation$Y11,  method = "pearson")
#Y9～11
cor.test(relation$Y9, relation$Y10,  method = "pearson")  
cor.test(relation$Y9, relation$Y11,  method = "pearson")
#Y10～11
cor.test(relation$Y10,relation$Y11,  method = "pearson")


	Pearson's product-moment correlation

data:  relation$Y1 and relation$Y2
t = 3.5192, df = 26, p-value = 0.001615
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.2473685 0.7765393
sample estimates:
      cor 
0.5680239 



	Pearson's product-moment correlation

data:  relation$Y1 and relation$Y3
t = -0.8296, df = 26, p-value = 0.4143
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.5034983  0.2260323
sample estimates:
       cor 
-0.1605865 



	Pearson's product-moment correlation

data:  relation$Y1 and relation$Y4
t = 0.37221, df = 26, p-value = 0.7128
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.3086585  0.4340886
sample estimates:
      cor 
0.0728018 



	Pearson's product-moment correlation

data:  relation$Y1 and relation$Y5
t = 1.6872, df = 26, p-value = 0.1035
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.06675679  0.61512802
sample estimates:
     cor 
0.314144 



	Pearson's product-moment correlation

data:  relation$Y1 and relation$Y6
t = -2.0161, df = 26, p-value = 0.05423
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.651410873  0.006239845
sample estimates:
       cor 
-0.3676931 


...


	Pearson's product-moment correlation

data:  relation$Y8 and relation$Y10
t = -0.96936, df = 26, p-value = 0.3413
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.5233726  0.2002684
sample estimates:
       cor 
-0.1867626 



	Pearson's product-moment correlation

data:  relation$Y8 and relation$Y11
t = -0.20713, df = 26, p-value = 0.8375
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.4074949  0.3376005
sample estimates:
        cor 
-0.04058856 



	Pearson's product-moment correlation

data:  relation$Y9 and relation$Y10
t = 0.58027, df = 26, p-value = 0.5667
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.2714573  0.4664701
sample estimates:
      cor 
0.1130708 



	Pearson's product-moment correlation

data:  relation$Y9 and relation$Y11
t = 0.53731, df = 26, p-value = 0.5956
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.2791975  0.4598918
sample estimates:
      cor 
0.1047951 



	Pearson's product-moment correlation

data:  relation$Y10 and relation$Y11
t = -2.0247, df = 26, p-value = 0.05328
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.652309740  0.004676858
sample estimates:
      cor 
-0.369044 


# 多変量時系列分析
- 単位根検定(ADF検定)
- 共和分検定
- 偏グレンジャー因果性検定と非直交化インパルス応答関数
- パネルVARモデルによる動的直接相関係数の導出
- 幾何ブラウン運動と確率偏微分方程式
- LSTM(Long Short Term Memory)

## 単位根検定(ADF検定)

In [9]:
relation %>%
  apply(2, ADF)

$Y1

############################################### 
# Augmented Dickey-Fuller Test Unit Root Test # 
############################################### 

Test regression drift 


Call:
lm(formula = z.diff ~ z.lag.1 + 1 + z.diff.lag)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.06802 -0.01276  0.00237  0.01493  0.05123 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept)  0.0004843  0.0052101   0.093  0.92674   
z.lag.1     -0.6311651  0.2209384  -2.857  0.00892 **
z.diff.lag  -0.0450053  0.1992156  -0.226  0.82326   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.02523 on 23 degrees of freedom
Multiple R-squared:  0.3465,	Adjusted R-squared:  0.2897 
F-statistic: 6.098 on 2 and 23 DF,  p-value: 0.0075


Value of test-statistic is: -2.8567 4.4176 

Critical values for test statistics: 
      1pct  5pct 10pct
tau2 -3.58 -2.93 -2.60
phi1  7.06  4.86  3.94


$Y2

####################################

## 共和分検定

In [10]:
#Y1～11
coint.test(relation$Y1, relation$Y2,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y3,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y4,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y5,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y6,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y7,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y8,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y1, relation$Y11, nlag = 1) %>% summary
#Y2～11
coint.test(relation$Y2, relation$Y3,  nlag = 1) %>% summary
coint.test(relation$Y2, relation$Y4,  nlag = 1) %>% summary
coint.test(relation$Y2, relation$Y6,  nlag = 1) %>% summary
coint.test(relation$Y2, relation$Y7,  nlag = 1) %>% summary
coint.test(relation$Y2, relation$Y8,  nlag = 1) %>% summary
coint.test(relation$Y2, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y2, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y2, relation$Y11, nlag = 1) %>% summary
#Y3～11
coint.test(relation$Y3, relation$Y4,  nlag = 1) %>% summary
coint.test(relation$Y3, relation$Y5,  nlag = 1) %>% summary
coint.test(relation$Y3, relation$Y6,  nlag = 1) %>% summary
coint.test(relation$Y3, relation$Y7,  nlag = 1) %>% summary
coint.test(relation$Y3, relation$Y8,  nlag = 1) %>% summary
coint.test(relation$Y3, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y3, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y3, relation$Y11, nlag = 1) %>% summary
#Y4～11
coint.test(relation$Y4, relation$Y5,  nlag = 1) %>% summary
coint.test(relation$Y4, relation$Y6,  nlag = 1) %>% summary
coint.test(relation$Y4, relation$Y7,  nlag = 1) %>% summary
coint.test(relation$Y4, relation$Y8,  nlag = 1) %>% summary
coint.test(relation$Y4, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y4, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y4, relation$Y11, nlag = 1) %>% summary
#Y5～11
coint.test(relation$Y5, relation$Y6,  nlag = 1) %>% summary
coint.test(relation$Y5, relation$Y7,  nlag = 1) %>% summary
coint.test(relation$Y5, relation$Y8,  nlag = 1) %>% summary
coint.test(relation$Y5, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y5, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y5, relation$Y11, nlag = 1) %>% summary
#Y6～11
coint.test(relation$Y6, relation$Y7,  nlag = 1) %>% summary
coint.test(relation$Y6, relation$Y8,  nlag = 1) %>% summary
coint.test(relation$Y6, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y6, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y6, relation$Y11, nlag = 1) %>% summary
#Y7～11
coint.test(relation$Y7, relation$Y8,  nlag = 1) %>% summary
coint.test(relation$Y7, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y7, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y7, relation$Y11, nlag = 1) %>% summary
#Y8～11
coint.test(relation$Y8, relation$Y9,  nlag = 1) %>% summary
coint.test(relation$Y8, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y8, relation$Y11, nlag = 1) %>% summary
#Y9～11
coint.test(relation$Y9, relation$Y10, nlag = 1) %>% summary
coint.test(relation$Y9, relation$Y11, nlag = 1) %>% summary
#Y10～11
coint.test(relation$Y10, relation$Y11, nlag = 1) %>% summary

Response: relation$Y1 
Input: relation$Y2 
Number of inputs: 1 
Model: y ~ X + 1 
------------------------------- 
Engle-Granger Cointegration Test 
alternative: cointegrated 

Type 1: no trend 
    lag      EG p.value 
   1.00   -4.55    0.01 
----- 
 Type 2: linear trend 
    lag      EG p.value 
   1.00   -1.44    0.10 
----- 
 Type 3: quadratic trend 
    lag      EG p.value 
  1.000   0.826   0.100 
----------- 
Note: p.value = 0.01 means p.value <= 0.01 
    : p.value = 0.10 means p.value >= 0.10 


      lag          EG             p.value     
 Min.   :1   Min.   :-4.5544   Min.   :0.010  
 1st Qu.:1   1st Qu.:-2.9967   1st Qu.:0.055  
 Median :1   Median :-1.4389   Median :0.100  
 Mean   :1   Mean   :-1.7225   Mean   :0.070  
 3rd Qu.:1   3rd Qu.:-0.3065   3rd Qu.:0.100  
 Max.   :1   Max.   : 0.8259   Max.   :0.100  

Response: relation$Y1 
Input: relation$Y3 
Number of inputs: 1 
Model: y ~ X + 1 
------------------------------- 
Engle-Granger Cointegration Test 
alternative: cointegrated 

Type 1: no trend 
    lag      EG p.value 
 1.0000 -3.3169  0.0308 
----- 
 Type 2: linear trend 
    lag      EG p.value 
   1.00   -1.54    0.10 
----- 
 Type 3: quadratic trend 
    lag      EG p.value 
   1.00    1.04    0.10 
----------- 
Note: p.value = 0.01 means p.value <= 0.01 
    : p.value = 0.10 means p.value >= 0.10 


      lag          EG             p.value       
 Min.   :1   Min.   :-3.3169   Min.   :0.03076  
 1st Qu.:1   1st Qu.:-2.4277   1st Qu.:0.06538  
 Median :1   Median :-1.5385   Median :0.10000  
 Mean   :1   Mean   :-1.2721   Mean   :0.07692  
 3rd Qu.:1   3rd Qu.:-0.2497   3rd Qu.:0.10000  
 Max.   :1   Max.   : 1.0391   Max.   :0.10000  

Response: relation$Y1 
Input: relation$Y4 
Number of inputs: 1 
Model: y ~ X + 1 
------------------------------- 
Engle-Granger Cointegration Test 
alternative: cointegrated 

Type 1: no trend 
    lag      EG p.value 
 1.0000 -3.2421  0.0349 
----- 
 Type 2: linear trend 
    lag      EG p.value 
   1.00   -1.52    0.10 
----- 
 Type 3: quadratic trend 
    lag      EG p.value 
  1.000   0.974   0.100 
----------- 
Note: p.value = 0.01 means p.value <= 0.01 
    : p.value = 0.10 means p.value >= 0.10 


...

      lag          EG             p.value     
 Min.   :1   Min.   :-4.9897   Min.   :0.010  
 1st Qu.:1   1st Qu.:-3.0290   1st Qu.:0.055  
 Median :1   Median :-1.0682   Median :0.100  
 Mean   :1   Mean   :-1.9485   Mean   :0.070  
 3rd Qu.:1   3rd Qu.:-0.4278   3rd Qu.:0.100  
 Max.   :1   Max.   : 0.2125   Max.   :0.100  

Response: relation$Y9 
Input: relation$Y11 
Number of inputs: 1 
Model: y ~ X + 1 
------------------------------- 
Engle-Granger Cointegration Test 
alternative: cointegrated 

Type 1: no trend 
    lag      EG p.value 
   1.00   -4.77    0.01 
----- 
 Type 2: linear trend 
    lag      EG p.value 
  1.000   0.219   0.100 
----- 
 Type 3: quadratic trend 
    lag      EG p.value 
   1.00   -1.05    0.10 
----------- 
Note: p.value = 0.01 means p.value <= 0.01 
    : p.value = 0.10 means p.value >= 0.10 


      lag          EG             p.value     
 Min.   :1   Min.   :-4.7739   Min.   :0.010  
 1st Qu.:1   1st Qu.:-2.9112   1st Qu.:0.055  
 Median :1   Median :-1.0485   Median :0.100  
 Mean   :1   Mean   :-1.8679   Mean   :0.070  
 3rd Qu.:1   3rd Qu.:-0.4148   3rd Qu.:0.100  
 Max.   :1   Max.   : 0.2188   Max.   :0.100  

Response: relation$Y10 
Input: relation$Y11 
Number of inputs: 1 
Model: y ~ X + 1 
------------------------------- 
Engle-Granger Cointegration Test 
alternative: cointegrated 

Type 1: no trend 
    lag      EG p.value 
   1.00   -4.42    0.01 
----- 
 Type 2: linear trend 
    lag      EG p.value 
  1.000   0.324   0.100 
----- 
 Type 3: quadratic trend 
    lag      EG p.value 
  1.000  -0.601   0.100 
----------- 
Note: p.value = 0.01 means p.value <= 0.01 
    : p.value = 0.10 means p.value >= 0.10 


      lag          EG             p.value     
 Min.   :1   Min.   :-4.4199   Min.   :0.010  
 1st Qu.:1   1st Qu.:-2.5104   1st Qu.:0.055  
 Median :1   Median :-0.6009   Median :0.100  
 Mean   :1   Mean   :-1.5657   Mean   :0.070  
 3rd Qu.:1   3rd Qu.:-0.1386   3rd Qu.:0.100  
 Max.   :1   Max.   : 0.3237   Max.   :0.100  

## 偏グレンジャー因果性検定と非直交化インパルス応答関数

In [11]:
#Y1～11
ts(relation$Y1, relation$Y2)
ts(relation$Y1, relation$Y3)
ts(relation$Y1, relation$Y4)
ts(relation$Y1, relation$Y5)
ts(relation$Y1, relation$Y6)
ts(relation$Y1, relation$Y7)
ts(relation$Y1, relation$Y8)
ts(relation$Y1, relation$Y9)
ts(relation$Y1, relation$Y10)
ts(relation$Y1, relation$Y11)
#Y2～11
ts(relation$Y2, relation$Y3)
ts(relation$Y2, relation$Y4)
ts(relation$Y2, relation$Y5)
ts(relation$Y2, relation$Y6)
ts(relation$Y2, relation$Y7)
ts(relation$Y2, relation$Y8)
ts(relation$Y2, relation$Y9)
ts(relation$Y2, relation$Y10)
ts(relation$Y2, relation$Y11)
#Y3～11
ts(relation$Y3, relation$Y4)
ts(relation$Y3, relation$Y5)
ts(relation$Y3, relation$Y6)
ts(relation$Y3, relation$Y7)
ts(relation$Y3, relation$Y8)
ts(relation$Y3, relation$Y9)
ts(relation$Y3, relation$Y10)
ts(relation$Y3, relation$Y11)
#Y4～11
ts(relation$Y4, relation$Y5)
ts(relation$Y4, relation$Y6)
ts(relation$Y4, relation$Y7)
ts(relation$Y4, relation$Y8)
ts(relation$Y4, relation$Y9)
ts(relation$Y4, relation$Y10)
ts(relation$Y4, relation$Y11)
#Y5～11
ts(relation$Y5, relation$Y6)
ts(relation$Y5, relation$Y7)
ts(relation$Y5, relation$Y8)
ts(relation$Y5, relation$Y9)
ts(relation$Y5, relation$Y10)
ts(relation$Y5, relation$Y11)
#Y6～11
ts(relation$Y6, relation$Y7)
ts(relation$Y6, relation$Y8)
ts(relation$Y6, relation$Y9)
ts(relation$Y6, relation$Y10)
ts(relation$Y6, relation$Y11)
#Y7～11
ts(relation$Y7, relation$Y8)
ts(relation$Y7, relation$Y9)
ts(relation$Y7, relation$Y10)
ts(relation$Y7, relation$Y11)
#Y8～11
ts(relation$Y8, relation$Y9)
ts(relation$Y8, relation$Y10)
ts(relation$Y8, relation$Y11)
#Y9～11
ts(relation$Y9, relation$Y10)
ts(relation$Y9, relation$Y11)
#Y10～11
ts(relation$Y10, relation$Y11)

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 2.6256, df1 = 2, df2 = 40, p-value = 0.08484


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 7.1644, df = 1, p-value = 0.007436



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 0.16329, df1 = 2, df2 = 40, p-value = 0.8499


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 7.1644, df = 1, p-value = 0.007436




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                y2
 [1,]  0.021746686
 [2,]  0.001309951
 [3,]  0.004524624
 [4,] -0.006752473
 [5,] -0.006897090
 [6,]  0.009214482
 [7,]  0.003973832
 [8,] -0.009230460
 [9,] -0.000667452
[10,]  0.007867168
[11,] -0.001802526


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.0000000000
 [2,]

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 0.74651, df1 = 2, df2 = 40, p-value = 0.4805


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 0.032198, df = 1, p-value = 0.8576



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 1.5637, df1 = 2, df2 = 40, p-value = 0.2219


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 0.032198, df = 1, p-value = 0.8576




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                 y2
 [1,] -9.040789e-03
 [2,] -3.784301e-02
 [3,]  1.911671e-03
 [4,]  3.692003e-02
 [5,] -2.074643e-02
 [6,] -6.640545e-03
 [7,]  1.093326e-02
 [8,] -4.518734e-03
 [9,]  1.391409e-03
[10,]  7.748545e-05
[11,] -2.040933e-03


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.00000

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 1.4966, df1 = 2, df2 = 40, p-value = 0.2362


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 1.5876, df = 1, p-value = 0.2077



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 0.77621, df1 = 2, df2 = 40, p-value = 0.467


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 1.5876, df = 1, p-value = 0.2077




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                y2
 [1,]  0.049693367
 [2,]  0.060200421
 [3,] -0.025674942
 [4,] -0.032149000
 [5,]  0.014708442
 [6,]  0.013562122
 [7,] -0.007651860
 [8,] -0.005308823
 [9,]  0.003744900
[10,]  0.001994856
[11,] -0.001769591


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.0000000000
 [2,] -0.00

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 0.77671, df1 = 2, df2 = 40, p-value = 0.4667


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 0.024608, df = 1, p-value = 0.8753



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 3.2556, df1 = 2, df2 = 40, p-value = 0.04898


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 0.024608, df = 1, p-value = 0.8753




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                 y2
 [1,]  2.448407e-03
 [2,] -3.870621e-03
 [3,] -1.900866e-02
 [4,] -2.815833e-04
 [5,]  1.401183e-02
 [6,]  2.688641e-03
 [7,] -5.233067e-03
 [8,] -2.122573e-03
 [9,] -3.214378e-05
[10,]  5.840611e-04
[11,]  1.470587e-03


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.0000

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 3.1308, df1 = 2, df2 = 40, p-value = 0.05455


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 6.8458, df = 1, p-value = 0.008885



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 1.1466, df1 = 2, df2 = 40, p-value = 0.328


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 6.8458, df = 1, p-value = 0.008885




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                y2
 [1,] -0.019739673
 [2,] -0.027218333
 [3,] -0.017310614
 [4,] -0.006807211
 [5,] -0.006123456
 [6,] -0.006468603
 [7,] -0.004636921
 [8,] -0.002690781
 [9,] -0.001999678
[10,] -0.001728475
[11,] -0.001294268


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.000000e+00
 [2,]  

...

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 4.2859, df1 = 2, df2 = 40, p-value = 0.02058


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 1.6725, df = 1, p-value = 0.1959



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 1.554, df1 = 2, df2 = 40, p-value = 0.2239


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 1.6725, df = 1, p-value = 0.1959




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                 y2
 [1,] -0.0645597949
 [2,] -0.0272612070
 [3,]  0.1227786054
 [4,]  0.0669878166
 [5,]  0.0314894809
 [6,] -0.0193043135
 [7,] -0.0199559070
 [8,] -0.0133848302
 [9,]  0.0006943244
[10,]  0.0045755677
[11,]  0.0044483816


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.000000e+00

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 4.1557, df1 = 2, df2 = 40, p-value = 0.02292


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 0.14797, df = 1, p-value = 0.7005



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 1.2625, df1 = 2, df2 = 40, p-value = 0.294


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 0.14797, df = 1, p-value = 0.7005




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                y2
 [1,] -0.015897912
 [2,]  0.147432315
 [3,] -0.144641097
 [4,]  0.046265389
 [5,]  0.017016557
 [6,] -0.059797930
 [7,]  0.063450346
 [8,] -0.031038984
 [9,] -0.003656091
[10,]  0.025321805
[11,] -0.028745615


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.000000e+00
 [2,]  1.

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 0.63316, df1 = 2, df2 = 40, p-value = 0.5361


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 0.59555, df = 1, p-value = 0.4403



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 1.0202, df1 = 2, df2 = 40, p-value = 0.3697


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 0.59555, df = 1, p-value = 0.4403




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                 y2
 [1,]  0.0436973192
 [2,]  0.0494480592
 [3,]  0.0350570171
 [4,] -0.0150019403
 [5,] -0.0062602053
 [6,]  0.0064019224
 [7,]  0.0005124702
 [8,] -0.0021392325
 [9,]  0.0003385096
[10,]  0.0005972819
[11,] -0.0002465640


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.0000000

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 1.6971, df1 = 2, df2 = 40, p-value = 0.1961


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 0.15945, df = 1, p-value = 0.6897



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 2.2174, df1 = 2, df2 = 40, p-value = 0.1221


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 0.15945, df = 1, p-value = 0.6897




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                y2
 [1,] -0.018159119
 [2,]  0.045711987
 [3,]  0.040821114
 [4,] -0.073999223
 [5,]  0.035983018
 [6,] -0.003276559
 [7,] -0.013012096
 [8,]  0.024414521
 [9,] -0.023673093
[10,]  0.010325595
[11,]  0.003860163


[[2]][[2]]

Impulse response coefficients
$y2
                 y1
 [1,]  0.0000000000
 [2,] -0.

[[1]]
[[1]][[1]]
[[1]][[1]]$Granger

	Granger causality H0: y1 do not Granger-cause y2

data:  VAR object model
F-Test = 0.033704, df1 = 2, df2 = 40, p-value = 0.9669


[[1]][[1]]$Instant

	H0: No instantaneous causality between: y1 and y2

data:  VAR object model
Chi-squared = 0.24038, df = 1, p-value = 0.6239



[[1]][[2]]
[[1]][[2]]$Granger

	Granger causality H0: y2 do not Granger-cause y1

data:  VAR object model
F-Test = 2.4985, df1 = 2, df2 = 40, p-value = 0.09496


[[1]][[2]]$Instant

	H0: No instantaneous causality between: y2 and y1

data:  VAR object model
Chi-squared = 0.24038, df = 1, p-value = 0.6239




[[2]]
[[2]][[1]]

Impulse response coefficients
$y1
                 y2
 [1,] -2.411141e-02
 [2,]  2.379493e-02
 [3,]  1.662515e-03
 [4,] -1.498251e-02
 [5,]  1.584177e-02
 [6,] -8.669036e-03
 [7,] -3.410830e-04
 [8,]  6.040227e-03
 [9,] -6.728445e-03
[10,]  3.814677e-03
[11,] -1.340125e-05


[[2]][[2]]

Impulse response coefficients
$y2
                y1
 [1,]  0.000000

## パネルVARモデルによる動的直接相関係数の導出
1. パネルデータの生成
1. 目視確認
1. 要約統計量を求める。
1. モデル形成
1. 詳細結果の目視確認
1. 動的直接相関係数の抽出
1. パネルVARモデルの標準誤差の抽出

In [12]:
#パネルデータの生成
index <- adjusted[, 1:2] %>% apply(2, as.character) %>% as.data.frame
panel <- bind_cols(index, relation) %>% pdata.frame(index = c("id", "time"))
#目視確認
panel

,id,time,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0-1987,0,1987,0.046607215,-0.0002075447,0.0,0.14329221,0.1707077926,-0.011862535,0.000000000,0.07410797,0.142432217,0.0000000000,0.0
0-1988,0,1988,0.054685886,0.0659405832,0.0,-0.27232247,0.1875249212,-0.036456042,0.006960585,0.00000000,0.328497543,0.0000000000,0.0
0-1989,0,1989,0.034524371,0.0250528768,0.0,0.19834206,0.0005498574,-0.042990185,0.022858138,-0.11332869,0.232060612,0.0000000000,0.0
0-1990,0,1990,0.038998803,0.0696837388,0.0,0.21128783,0.0213535400,-0.050341755,0.030052345,-0.08338161,-0.519724354,-0.0870144800,0.0
0-1991,0,1991,0.014394272,0.0108689107,-0.5,-0.15698568,0.1077701561,-0.016438726,0.033426293,-0.09097178,-0.070368106,0.0000000000,0.0
0-1992,0,1992,0.008714722,0.0088388117,0.5,-0.03704127,0.0555237206,0.019152432,0.015781495,0.00000000,-0.321780945,-0.2076436613,-0.5
0-1993,0,1993,0.010772766,0.0018240989,0.0,-0.15539469,0.1027744168,0.078164773,0.013478690,0.04652002,0.015192965,0.0000000000,0.5
0-1994,0,1994,0.034959424,0.0388164337,0.0,-0.02806837,0.0898898827,0.029631798,0.006160184,0.12783337,0.118167806,0.0000000000,-0.5
0-1995,0,1995,0.031159255,0.0255087629,-0.5,-0.01487016,0.0913039761,0.059049029,-0.001024066,0.14842001,0.008353503,0.0000000000,0.5


In [13]:
#要約統計量を求める。
panel %>%
  summary

 id          time          Y1                  Y2                  Y3      
 0:24   1987   : 1   Min.   :-0.072977   Min.   :-0.116774   Min.   :-0.5  
 1: 4   1988   : 1   1st Qu.:-0.012895   1st Qu.:-0.007578   1st Qu.: 0.0  
        1989   : 1   Median : 0.009726   Median : 0.006923   Median : 0.0  
        1990   : 1   Mean   : 0.006292   Mean   : 0.004733   Mean   : 0.0  
        1991   : 1   3rd Qu.: 0.026149   3rd Qu.: 0.025167   3rd Qu.: 0.0  
        1992   : 1   Max.   : 0.054686   Max.   : 0.069684   Max.   : 0.5  
        (Other):22                                                         
       Y4                 Y5                 Y6                 Y7           
 Min.   :-0.38750   Min.   :-0.18289   Min.   :-0.05034   Min.   :-0.014301  
 1st Qu.:-0.04347   1st Qu.:-0.04042   1st Qu.: 0.01807   1st Qu.:-0.002306  
 Median : 0.03837   Median : 0.04660   Median : 0.04968   Median : 0.000512  
 Mean   : 0.05178   Mean   : 0.02571   Mean   : 0.04501   Mean   : 0.005141  
 3

In [14]:
#モデルの形成
model <- pvargmm(dependent_vars = c("Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10", "Y11"), 
                 lags = 1,
                 transformation = c("fod"),
                 data = panel,
                 panel_identifier =  c("id", "time"), 
                 steps = c("twostep"),
                 system_instruments = F,
                 max_instr_dependent_vars = 2,
                 min_instr_dependent_vars = 1L
)

Warning message in pvargmm(dependent_vars = c("Y1", "Y2", "Y3", "Y4", "Y5", "Y6", :
"The matrix Lambda is singular, therefore the general inverse is used"

 One-step estimation: Matrix inversion
                                                                              

Warning message in pvargmm(dependent_vars = c("Y1", "Y2", "Y3", "Y4", "Y5", "Y6", :
"The matrix D_e is singular, therefore the general inverse is used"
Warning message in sqrt(diag(var_first_step)):
" 計算結果が NaN になりました "

Windmeijer - Sigmund robust se: [===========================] Iteration 2 of 2
                                                                              



In [15]:
#詳細結果の目視確認
model %>%
  summary

---------------------------------------------------
Dynamic Panel VAR estimation, two-step GMM 
---------------------------------------------------
Transformation: Forward orthogonal deviations 
Group variable: id 
Time variable: time 
Number of observations = 16 
Number of groups = 2 
Obs per group: min = 0 
               avg = 8 
               max = 16 
Number of instruments = 6171 

          Y1        Y2        Y3        Y4        Y5        Y6        Y7        Y8        Y9        Y10       Y11     
----------------------------------------------------------------------------------------------------------------------
lag1_Y1   -0.0028   -0.0062    0.2220   -0.0480   -0.0713    0.0020   -0.0032    0.0356   -0.2040    0.0030    0.0694 
          (0.0029)  (0.0064)  (0.2297)  (0.0497)  (0.0737)  (0.0021)  (0.0033)  (0.0368)  (0.2110)  (0.0031)  (0.0718)
lag1_Y2   -0.0050    0.0275    0.0665   -0.0711    0.0470    0.0084    0.0002    0.0002    0.0185    0.3901   -0.0490 
          (0.0

In [16]:
#動的直接相関係数の抽出
coefficient <- model %>%
  coef %>%
  apply(2, as.numeric) %>%
  as.data.frame
coefficient

fod_lag1_Y1,fod_lag1_Y2,fod_lag1_Y3,fod_lag1_Y4,fod_lag1_Y5,fod_lag1_Y6,fod_lag1_Y7,fod_lag1_Y8,fod_lag1_Y9,fod_lag1_Y10,fod_lag1_Y11
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.002755249,-0.0050207787,-0.16931255,-0.101768112,0.0009635997,0.018466039,-0.0002774653,0.009914154,0.01369006,-0.077207792,0.09705778
-0.006195090,0.0275329642,-0.09964652,-0.203052304,-0.0201305406,0.015078743,0.0018038768,-0.045506299,-0.01724720,0.137895850,0.01045967
0.222002529,0.0665407711,1.13272639,0.141606922,0.2713863935,-0.451975130,0.0510547863,0.157144539,0.15545010,-1.515287824,-0.27741172
-0.048013232,-0.0711160547,0.38291057,0.421612797,0.1218261696,0.036098516,-0.0135971765,0.340138699,-1.09443628,-0.435713156,-0.05399573
-0.071277637,0.0469653214,-0.90920739,-0.785444930,-0.1987902325,0.113329527,0.0152993784,-0.218143602,0.29830157,0.317430644,-0.10862941
0.002014497,0.0083530692,0.07643506,0.073981297,0.0154727523,-0.006572534,-0.0010711035,-0.004921998,-0.01333021,0.068395469,-0.10447770
-0.003231988,0.0002458802,0.01588686,-0.007247741,-0.0015249092,-0.001877827,0.0011170139,-0.001197610,-0.02346839,0.002867628,-0.01703419
0.035585016,0.0001843510,-0.62833666,-0.381344959,0.0305652783,0.021040742,-0.0023231949,0.035958589,0.29516954,-0.468530403,0.33108238
-0.203964391,0.0184871855,-0.13789203,-0.713690182,-0.2511128046,-0.035476207,0.0391586341,-0.851447218,1.52166575,1.620152172,-0.96231174


In [17]:
#要約統計量を求める。
coefficient %>%
  summary

  fod_lag1_Y1          fod_lag1_Y2         fod_lag1_Y3        fod_lag1_Y4      
 Min.   :-0.2039644   Min.   :-0.071116   Min.   :-3.09320   Min.   :-1.13385  
 1st Qu.:-0.0271042   1st Qu.:-0.002418   1st Qu.:-0.39882   1st Qu.:-0.54752  
 Median :-0.0027552   Median : 0.008353   Median :-0.09965   Median :-0.20305  
 Mean   :-0.0003204   Mean   : 0.039390   Mean   :-0.20253   Mean   :-0.27884  
 3rd Qu.: 0.0192711   3rd Qu.: 0.037249   3rd Qu.: 0.22967   3rd Qu.: 0.03337  
 Max.   : 0.2220025   Max.   : 0.390075   Max.   : 1.20177   Max.   : 0.42161  
  fod_lag1_Y5          fod_lag1_Y6        fod_lag1_Y7       
 Min.   :-0.2511128   Min.   :-0.45198   Min.   :-0.044992  
 1st Qu.:-0.0848997   1st Qu.:-0.02102   1st Qu.:-0.001697  
 Median : 0.0009636   Median : 0.01508   Median : 0.001117  
 Mean   : 0.0181746   Mean   :-0.02097   Mean   : 0.008900  
 3rd Qu.: 0.0761957   3rd Qu.: 0.02857   3rd Qu.: 0.027229  
 Max.   : 0.3809336   Max.   : 0.34086   Max.   : 0.051723  
  fod_lag1_Y8

In [18]:
#モデルの標準誤差
SE <- model %>%
  se %>%
  apply(2, as.numeric) %>%
  as.data.frame
SE

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.002850591,0.0051945161,0.1751714,0.10528966,0.0009969438,0.019105032,0.0002870666,0.010257220,0.01416379,0.079879466,0.10041634
0.006409463,0.0284857061,0.1030947,0.21007866,0.0208271314,0.015600523,0.0018662976,0.047080984,0.01784402,0.142667554,0.01082161
0.229684634,0.0688433269,1.1719229,0.14650704,0.2807773622,0.467615134,0.0528214700,0.162582319,0.16082925,1.567722364,0.28701119
0.049674667,0.0735769322,0.3961607,0.43620215,0.1260418038,0.037347657,0.0140676889,0.351908751,1.13230781,0.450790436,0.05586418
0.073744106,0.0485904945,0.9406693,0.81262422,0.2056691068,0.117251146,0.0158287932,0.225692174,0.30862390,0.328414913,0.11238839
0.002084206,0.0086421162,0.0790800,0.07654132,0.0160081665,0.006799968,0.0011081676,0.005092317,0.01379148,0.070762204,0.10809301
0.003343827,0.0002543886,0.0164366,0.00749854,0.0015776767,0.001942807,0.0011556667,0.001239051,0.02428048,0.002966858,0.01762364
0.036816389,0.0001907302,0.6500794,0.39454090,0.0316229495,0.021768829,0.0024035860,0.037202888,0.30538349,0.484743280,0.34253905
0.211022310,0.0191269102,0.1426636,0.73838649,0.2598022325,0.036703814,0.0405136671,0.880910427,1.57432099,1.676215404,0.99561127


In [20]:
#要約統計量を求める。
SE %>%
  summary

       V1                 V2                  V3                V4          
 Min.   :0.002084   Min.   :0.0001907   Min.   :0.01644   Min.   :0.007498  
 1st Qu.:0.003202   1st Qu.:0.0069183   1st Qu.:0.12288   1st Qu.:0.125898  
 Median :0.036816   Median :0.0284857   Median :0.39616   Median :0.391145  
 Mean   :0.062768   Mean   :0.0642843   Mean   :0.73808   Mean   :0.408355  
 3rd Qu.:0.072749   3rd Qu.:0.0597461   3rd Qu.:1.05630   3rd Qu.:0.587294  
 Max.   :0.229685   Max.   :0.4035734   Max.   :3.20023   Max.   :1.173088  
       V5                  V6                 V7                  V8          
 Min.   :0.0009969   Min.   :0.001943   Min.   :0.0002871   Min.   :0.001239  
 1st Qu.:0.0184176   1st Qu.:0.017353   1st Qu.:0.0015110   1st Qu.:0.023730  
 Median :0.1260418   Median :0.036704   Median :0.0140677   Median :0.162582  
 Mean   :0.1356624   Mean   :0.124192   Mean   :0.0209195   Mean   :0.272398  
 3rd Qu.:0.2327357   3rd Qu.:0.203289   3rd Qu.:0.0435314   3rd Qu

## 幾何ブラウン運動
1. パラメータ設定
1. 各列にシミュレートする。
1. その確率ベクトルを横結合し、確率偏微分方程式を形成する。
1. 結果の目視確認

### パラメータ設定
1. 標準偏回帰係数
1. ドリフト項
1. ボラティリティ
1. シミュレーションの設定

In [21]:
#標準偏回帰係数
SE_coefficient <- coefficient / SE %>%
  apply(2, as.numeric) %>%
  as.data.frame
#ドリフト項
mean_coefficient <- SE_coefficient %>%
  apply(2, mean) %>%
  as.data.frame
#ボラティリティ
volatility_coefficient <- SE_coefficient %>%
  apply(2, sigma) %>%
  as.data.frame
#シミュレーションの設定
N  <- 2050 - 1987 %>% as.numeric #標本数
t0 <- 1987 %>% as.numeric #開始時(年)
T  <- 2050 %>% as.numeric #終了時(年)

### パラメータ抽出
1. ドリフト項
1. ボラティリティ

In [22]:
#ドリフト項
mean_Y1  <- mean_coefficient[1, ]  %>% as.numeric
mean_Y2  <- mean_coefficient[2, ]  %>% as.numeric
mean_Y3  <- mean_coefficient[3, ]  %>% as.numeric
mean_Y4  <- mean_coefficient[4, ]  %>% as.numeric
mean_Y5  <- mean_coefficient[5, ]  %>% as.numeric
mean_Y6  <- mean_coefficient[6, ]  %>% as.numeric
mean_Y7  <- mean_coefficient[7, ]  %>% as.numeric
mean_Y8  <- mean_coefficient[8, ]  %>% as.numeric
mean_Y9  <- mean_coefficient[9, ]  %>% as.numeric
mean_Y10 <- mean_coefficient[10, ] %>% as.numeric
mean_Y11 <- mean_coefficient[11, ] %>% as.numeric
#ボラティリティ
volatility_Y1  <- volatility_coefficient[1, ]  %>% as.numeric
volatility_Y2  <- volatility_coefficient[2, ]  %>% as.numeric
volatility_Y3  <- volatility_coefficient[3, ]  %>% as.numeric
volatility_Y4  <- volatility_coefficient[4, ]  %>% as.numeric
volatility_Y5  <- volatility_coefficient[5, ]  %>% as.numeric
volatility_Y6  <- volatility_coefficient[6, ]  %>% as.numeric
volatility_Y7  <- volatility_coefficient[7, ]  %>% as.numeric
volatility_Y8  <- volatility_coefficient[8, ]  %>% as.numeric
volatility_Y9  <- volatility_coefficient[9, ]  %>% as.numeric
volatility_Y10 <- volatility_coefficient[10, ] %>% as.numeric
volatility_Y11 <- volatility_coefficient[11, ] %>% as.numeric

### 幾何ブラウン運動の実行
- Y1~11について時間微分係数の期待値を求める。
- 各列について実行する。

In [23]:
#時間微分係数の期待値を求める。
SDE_Y1  <- GBM(N = N, t0 = t0, theta = mean_Y1,  sigma = volatility_Y1,  T = T)  #Y1
SDE_Y2  <- GBM(N = N, t0 = t0, theta = mean_Y2,  sigma = volatility_Y2,  T = T)  #Y2
SDE_Y3  <- GBM(N = N, t0 = t0, theta = mean_Y3,  sigma = volatility_Y3,  T = T)  #Y3
SDE_Y4  <- GBM(N = N, t0 = t0, theta = mean_Y4,  sigma = volatility_Y4,  T = T)  #Y4
SDE_Y5  <- GBM(N = N, t0 = t0, theta = mean_Y5,  sigma = volatility_Y5,  T = T)  #Y5
SDE_Y6  <- GBM(N = N, t0 = t0, theta = mean_Y6,  sigma = volatility_Y6,  T = T)  #Y6
SDE_Y7  <- GBM(N = N, t0 = t0, theta = mean_Y7,  sigma = volatility_Y7,  T = T)  #Y7
SDE_Y8  <- GBM(N = N, t0 = t0, theta = mean_Y8,  sigma = volatility_Y8,  T = T)  #Y8
SDE_Y9  <- GBM(N = N, t0 = t0, theta = mean_Y9,  sigma = volatility_Y9,  T = T)  #Y9
SDE_Y10 <- GBM(N = N, t0 = t0, theta = mean_Y10, sigma = volatility_Y10, T = T)  #Y10
SDE_Y11 <- GBM(N = N, t0 = t0, theta = mean_Y11, sigma = volatility_Y11, T = T)  #Y11

### 確率偏微分方程式の形成
1. 各列に実行した幾何ブラウン運動の結果を横結合する。
1. 結果の目視確認を行う。
1. 要約統計量を求める。

In [24]:
#横結合
SPDE <- bind_cols(
    SDE_Y1
    , SDE_Y2
    , SDE_Y3
    , SDE_Y4
    , SDE_Y5
    , SDE_Y6
    , SDE_Y7
    , SDE_Y8
    , SDE_Y9
    , SDE_Y10
    , SDE_Y11
) %>%
  apply(2, as.numeric) %>%
  as.data.frame
#列名変更
colnames(SPDE) <- colnames(relation)
#目視確認
SPDE

New names:
* NA -> ...1
* NA -> ...2
* NA -> ...3
* NA -> ...4
* NA -> ...5
* ...



Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,1.214194e+142,0,0,2.787757e-295,2.787757e-295,2.787757e-295,0,6.826907e-96,2.787757e-295,0
0,2.419768e+141,0,0,4.684173e-295,1.113492e-295,5.205123e-296,0,3.505132e-96,5.451736e-296,0
0,2.156236e+141,0,0,4.017149e-295,2.810468e-296,5.590440e-296,0,4.493614e-96,1.323523e-295,0
0,5.010521e+141,0,0,4.009336e-295,3.497953e-296,1.103308e-295,0,2.953780e-96,2.212361e-295,0
0,1.050002e+142,0,0,9.896845e-296,3.181658e-296,6.637777e-296,0,4.605289e-96,5.444753e-296,0
0,1.141174e+142,0,0,2.883369e-296,6.612761e-296,1.202697e-296,0,1.767818e-96,7.421686e-296,0
0,3.504441e+142,0,0,4.233560e-297,5.286265e-296,9.311279e-298,0,3.816618e-97,6.733747e-296,0
0,2.607086e+142,0,0,2.168313e-297,1.714583e-296,2.603109e-298,0,1.195464e-96,7.131025e-296,0
0,1.837720e+142,0,0,6.367718e-298,4.050512e-296,7.346501e-299,0,5.561352e-97,1.207787e-295,0


In [25]:
#要約統計量を求める。
SPDE %>%
  summary

       Y1          Y2                   Y3          Y4          Y5            
 Min.   :0   Min.   :2.156e+141   Min.   :0   Min.   :0   Min.   : 0.000e+00  
 1st Qu.:0   1st Qu.:2.275e+143   1st Qu.:0   1st Qu.:0   1st Qu.: 0.000e+00  
 Median :0   Median :6.787e+144   Median :0   Median :0   Median :1.000e-301  
 Mean   :0   Mean   :4.501e+146   Mean   :0   Mean   :0   Mean   :2.636e-296  
 3rd Qu.:0   3rd Qu.:7.361e+145   3rd Qu.:0   3rd Qu.:0   3rd Qu.:4.350e-299  
 Max.   :0   Max.   :8.445e+147   Max.   :0   Max.   :0   Max.   :4.684e-295  
       Y6                   Y7                   Y8          Y9           
 Min.   :1.200e-301   Min.   : 0.000e+00   Min.   :0   Min.   :2.490e-97  
 1st Qu.:1.555e-298   1st Qu.: 0.000e+00   1st Qu.:0   1st Qu.:2.892e-96  
 Median :7.707e-297   Median :5.000e-302   Median :0   Median :6.639e-96  
 Mean   :2.862e-296   Mean   :9.015e-297   Mean   :0   Mean   :1.351e-95  
 3rd Qu.:3.373e-296   3rd Qu.:9.980e-300   3rd Qu.:0   3rd Qu.:1.944e-95

## LSTM(Long Short Term Memory)
- 時系列予測の精度評価を行う。
- 時系列データ向けのDeepLearningである。

### 出力保存先の相対パス指定

In [26]:
setwd("./1_output")

### 関数定義の実行
- 確率ベクトル毎に実行する。
- LSTM(Long Short Term Memory)による幾何ブラウン運動の精度評価
- 値が0以外の確率過程について精度評価を行う。
- グラフ描画も関数定義に含む。

In [ ]:
# LSTM(SDE_Y1,  10)
LSTM(SDE_Y2,  10)
ggsave("LSTM_Y2.jpg")
# LSTM(SDE_Y3,  10)
# LSTM(SDE_Y4,  10)
LSTM(SDE_Y5,  10)
ggsave("LSTM_Y5.jpg")
LSTM(SDE_Y6,  10)
ggsave("LSTM_Y6.jpg")
LSTM(SDE_Y7,  10)
ggsave("LSTM_Y7.jpg")
# LSTM(SDE_Y8,  10)
LSTM(SDE_Y9,  10)
ggsave("LSTM_Y9.jpg")
LSTM(SDE_Y10, 10)
ggsave("LSTM_Y10.jpg")
# LSTM(SDE_Y11, 10)

### 結果.csvの出力

In [ ]:
fwrite(SPDE, "panel_VAR_SPDE_LSTM_model.csv")

# グラフ描画・出力
1. ローデータの多変量時系列プロット
1. 誤差項調整した多変量時系列プロット
1. 相関行列の可視化による多重共線性の目視確認
1. 幾何ブラウン運動の多変量時系列プロット

## ローデータの多変量時系列プロット

In [ ]:
#各グラフの作成
fig_1  <- fig(raw_data, raw_data$Y1, "最終エネルギー消費量(EJ)", "統計値")
fig_2  <- fig(raw_data, raw_data$Y2, "温室効果ガス排出量(t-CO2換算)", "統計値")
fig_3  <- fig(raw_data, raw_data$Y3,  "原発事故・異常事象(有無)", "統計値")
fig_4  <- fig(raw_data, raw_data$Y4, "原油平均価格(実質US$/bbl)", "統計値")
fig_5  <- fig(raw_data, raw_data$Y5, "実質GDP(US$/消費者物価指数: 2015年 = 1)", "統計値")
fig_6  <- fig(raw_data, raw_data$Y6, "GDP比国債残高(%)", "統計値")
fig_7  <- fig(raw_data, raw_data$Y7, "年間平均消費者物価指数(2015年 = 100)", "統計値")
fig_8  <- fig(raw_data, raw_data$Y8, "年間平均完全失業率(%)","統計値")
fig_9  <- fig(raw_data, raw_data$Y9, "日経平均株価終値(実質円)", "統計値")
fig_10 <- fig(raw_data, raw_data$Y10, "自民党の衆院議席獲得率(%)", "統計値")
fig_11 <- fig(raw_data, raw_data$Y11, "気候変動対策に関する合意(有無)", "統計値")
fig_12 <- fig(raw_data, raw_data$id,  "エネルギー基本計画の策定・改正(有無)", "統計値")
#一枚に集約して出力する。
grid.arrange(fig_1, fig_2, fig_3, fig_4, fig_5, fig_6, fig_7, fig_8, fig_9, fig_10, fig_11, fig_12)
ggsave("Multivariate_Time_Series_Plot_raw_data.jpg")

## 誤差項調整した多変量時系列プロット

In [ ]:
#各グラフの作成
fig_1  <- fig(panel, panel$Y1,  "最終エネルギー消費量(EJ)", "対数差分系列")
fig_2  <- fig(panel, panel$Y2,  "温室効果ガス排出量(t-CO2換算)", "対数差分系列")
fig_3  <- fig(panel, panel$Y3,  "原発事故・異常事象(有無)", "対数差分プロビット写像")
fig_4  <- fig(panel, panel$Y4,  "原油平均価格(実質US$/bbl)", "対数差分系列")
fig_5  <- fig(panel, panel$Y5,  "実質GDP(US$/消費者物価指数: 2015年 = 1)", "対数差分系列")
fig_6  <- fig(panel, panel$Y6,  "GDP比国債残高(%)", "対数差分系列")
fig_7  <- fig(panel, panel$Y7,  "年間平均消費者物価指数(2015年 = 100)","対数差分系列")
fig_8  <- fig(panel, panel$Y8,  "年間平均完全失業率(%)", "対数差分系列")
fig_9  <- fig(panel, panel$Y9,  "日経平均株価終値(実質円)", "対数差分系列")
fig_10 <- fig(panel, panel$Y10, "自民党の衆院議席獲得率(%)", "対数差分系列")
fig_11 <- fig(panel, panel$Y11, "気候変動対策に関する合意(有無)", "対数差分プロビット写像")
fig_12 <- fig(panel, panel$id,  "エネルギー基本計画の策定・改正(有無)", "対数差分系列")
#一枚に集約して出力する。
grid.arrange(fig_1, fig_2, fig_3, fig_4, fig_5, fig_6, fig_7, fig_8, fig_9, fig_10, fig_11, fig_12)
ggsave("Multivariate_Time_Series_Plot_adjusted.jpg")

## 相関行列の可視化による多重共線性の目視確認

In [ ]:
qgraph(cor(relation), edge.labels = TRUE)
ggsave("relation_vector.jpg")

## 幾何ブラウン運動の多変量時系列プロット

In [ ]:
#classをtsからdata.frameに変換し、グラフの規格化を図る。
plot_SDE_Y1  <- SDE_Y1  %>% ts_df
plot_SDE_Y2  <- SDE_Y2  %>% ts_df
plot_SDE_Y3  <- SDE_Y3  %>% ts_df
plot_SDE_Y4  <- SDE_Y4  %>% ts_df
plot_SDE_Y5  <- SDE_Y5  %>% ts_df
plot_SDE_Y6  <- SDE_Y6  %>% ts_df
plot_SDE_Y7  <- SDE_Y7  %>% ts_df
plot_SDE_Y8  <- SDE_Y8  %>% ts_df
plot_SDE_Y9  <- SDE_Y9  %>% ts_df
plot_SDE_Y10 <- SDE_Y10 %>% ts_df
plot_SDE_Y11 <- SDE_Y11 %>% ts_df
#各グラフの作成
fig_1  <- fig(plot_SDE_Y1,plot_SDE_Y1$value,   "最終エネルギー消費量(EJ)","時間微分係数の期待値")
fig_2  <- fig(plot_SDE_Y2,plot_SDE_Y2$value,   "温室効果ガス排出量(t-CO2換算)","時間微分係数の期待値")
fig_3  <- fig(plot_SDE_Y3,plot_SDE_Y3$value,   "原発事故・異常事象(有無)","時間微分係数の期待値")
fig_4  <- fig(plot_SDE_Y4,plot_SDE_Y4$value,   "原油平均価格(実質US$/bbl)","時間微分係数の期待値")
fig_5  <- fig(plot_SDE_Y5,plot_SDE_Y5$value,   "実質GDP(US$/消費者物価指数: 2015年 = 1)","時間微分係数の期待値")
fig_6  <- fig(plot_SDE_Y6,plot_SDE_Y6$value,   "GDP比国債残高(%)","時間微分係数の期待値")
fig_7  <- fig(plot_SDE_Y7,plot_SDE_Y7$value,   "年間平均消費者物価指数(2015年 = 100)","時間微分係数の期待値")
fig_8  <- fig(plot_SDE_Y8,plot_SDE_Y8$value,   "年間平均完全失業率(%)","時間微分係数の期待値")
fig_9  <- fig(plot_SDE_Y9,plot_SDE_Y9$value,   "日経平均株価終値(実質円)","時間微分係数の期待値")
fig_10 <- fig(plot_SDE_Y10,plot_SDE_Y10$value, "自民党の衆院議席獲得率(%)","時間微分係数の期待値")
fig_11 <- fig(plot_SDE_Y11,plot_SDE_Y11$value, "気候変動対策に関する合意(有無)","時間微分係数の期待値")
#一枚に集約して出力する。
grid.arrange(fig_1, fig_2, fig_3, fig_4, fig_5, fig_6, fig_7, fig_8, fig_9, fig_10, fig_11)
ggsave("Multivariate_Time_Series_Plot_GBM.jpg")